# Reconocedor de lenguaje de señas Argentino entrenado solo con el dataset argentino.

In [17]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

Cargamos el modelo de manera que sea entrenable. No incluímos su última capa para poder establecer nuestras propias clases.

Importamos los datasets y hacemos un split.
Comencemos por el dataset mas populado, el de lenguaje de señas americano, que utilizaremos para entrenar las capas intermedias.


In [19]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Definimos los parametros
image_size = (299, 299)
batch_size = 32
asl_dir = "asl_dataset/"
train_val_seed = 42        # Es importante que sea la misma para ambos llamados

# Y creamos los conjuntos de entrenamiento y validacion. 
# Esto es medio raro, porque invocamos dos veces a image_dataset_from_directory para hacer el split,
# pero es la manera que indica la documentacion
asl_train_ds = image_dataset_from_directory(
    asl_dir,
    validation_split=0.2,
    subset="training",
    seed=train_val_seed, 
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int'   # or 'categorical' if you want one-hot
)

asl_val_ds = image_dataset_from_directory(
    asl_dir,
    validation_split=0.2,
    subset="validation",
    seed=train_val_seed, 
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int'
)

class_names = asl_train_ds.class_names

Found 2515 files belonging to 36 classes.
Using 2012 files for training.
Found 2515 files belonging to 36 classes.
Using 503 files for validation.


Procesamos ahora las imagenes para adecuarlas al formato de *InceptionV3*,

In [20]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

def preprocess_img(image, label):
    image = preprocess_input(image) 
    return image, label

asl_train_ds = asl_train_ds.map(preprocess_img).prefetch(tf.data.AUTOTUNE)
asl_val_ds   = asl_val_ds.map(preprocess_img).prefetch(tf.data.AUTOTUNE)

Chequeemos que obtuvimos las clases correctas,

In [21]:
print(class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Continuemos con la carga del dataset de lenguaje de señas argentino que definira las clases sobre la que predecirá el modelo.

In [22]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Extraemos los nombres de todas las imagenes que vamos a utilizar
lsa_dir = 'lsa16_segmented/'
filenames = [f for f in os.listdir(image_dir)]

# Y de cada una extraemos su clase, que viene dada por el primer numero del nombre
labels = [int(f.split('_')[0]) - 1 for f in filenames]   # Le restamos 1 a los labels para que esten en rango [0, 16) en vez de [1, 16]

# Y creamos un dataframe que asocia a cada nombre de archivo su clase.
lsa_df = pd.DataFrame({'filename': filenames, 'class': labels})

Preprocesamos las imagenes para adecuarlas al formato de ImageNet

In [23]:
# Separamos al dataset en train y validacion.
lsa_train_df, lsa_val_df = train_test_split(lsa_df,
                                    test_size=0.2,
                                    stratify=df['class'],   # Hace que se mantengan las proporciones de las clases luego del split
                                    random_state=42)

Creamos un *pipeline* de datos de *TensorFlow*. La idea es aprovechar la paralelización del *map* para procesar los datos mas rápido.

In [24]:
# Definimos una funcion que dado un filename devuelve su imagen y su clase o label
def load_and_preprocess(image_path, label):

    # Leemos el archivo y lo decodificamos en RGB
    img = tf.io.read_file(image_dir + image_path)
    img = tf.image.decode_jpeg(img, channels=3) 
    
    # Lo preprocesamos para InceptionV3
    img = tf.image.resize(img, [299, 299])
    img = preprocess_input(img)  # Obs. que preprocess_input es una funcion de inception_v3 en particular
    
    return img, label

# Usamos un batch_size de TensorFlow estandar
batch_size = 32

# 1. Cargamos el dataframe
lsa_ds = tf.data.Dataset.from_tensor_slices((lsa_train_df['filename'].values, lsa_train_df['class'].values))

# 2. Le mappeamos el preprocesamiento a cada entrada, paralelizando
lsa_ds = lsa_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# 3. Mezclamos para randomizar el orden de las muestras
lsa_ds = lsa_ds.shuffle(buffer_size=len(lsa_train_df))

# 4. Usamos el batch_size estandar
lsa_ds = lsa_ds.batch(batch_size)

# 5. Permitimos el prefetching del proximo batch
lsa_train_ds = lsa_ds.prefetch(tf.data.AUTOTUNE)                                                       

# Y repetimos lo mismo para el conjunto de validacion
lsa_val_ds = tf.data.Dataset.from_tensor_slices((lsa_val_df['filename'].values, lsa_val_df['class'].values))\
           .map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
           .batch(batch_size) \
           .prefetch(tf.data.AUTOTUNE)

Entrenemos ahora las capas intermedias del modelo con el *dataset* de ASL,

In [27]:
# Cargamos InceptionV3
base_model = InceptionV3(weights = 'imagenet',       # Pre-entrenado con ImageNet
                         include_top = False,        # Sin incluir su capa de clasificacion con 1000 clases para poder hacer fine-tuning 
                         input_shape = (299, 299, 3) # Necesario cuando no incluimos la ultima capa
                        )

# Inicialmente descongelamos todas las capas, despues congelamos las que no queremos que se entrenen
base_model.trainable = True

# Descongelamos desde la capa llamada mixed7, lo que descongela las ultimas ~50 capas.
set_trainable = False
for layer in base_model.layers:
    if layer.name == "mixed7":
        set_trainable = True
    layer.trainable = set_trainable

# Construimos la cabeza de clasificacion para las 36 clases de ASL
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(36, activation='softmax')(x)  # 26 letras + 10 digitos

model = Model(inputs=base_model.input, outputs=predictions)

# Compilamos el modelo usando un learning_rate bajo.
tuning_learning_rate = 1e-5
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=tuning_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Lo entrenamos con esos datos
model.fit(asl_train_ds, validation_data=asl_val_ds, epochs=5)

# Y nos guardamos los pesos del modelo del cual luego usaremos todo menos la cabeza de clasificacion.
model.save_weights("inceptionv3_hand_features.weights.h5")

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.0514 - loss: 3.6089 - val_accuracy: 0.1590 - val_loss: 3.3617
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.3401 - loss: 2.9007 - val_accuracy: 0.5706 - val_loss: 2.6478
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 190s 3s/step - accuracy: 0.6118 - loss: 2.2717 - val_accuracy: 0.8310 - val_loss: 1.8617
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 174s 3s/step - accuracy: 0.7797 - loss: 1.6661 - val_accuracy: 0.9125 - val_loss: 1.2535
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 172s 3s/step - accuracy: 0.8633 - loss: 1.2156 - val_accuracy: 0.9423 - val_loss: 0.8423


Ahora cargamos ese modelo que entrenamos pero le sacamos la cabeza y colocamos la clasificadora de LSA.

In [30]:
# Reconstruimos el modelo, nuevamente sin incluir el top.
base_model = InceptionV3(weights=None, include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # Y en este caso freezamos todas las capas pues solo queremos entrenar la que agregaremos

# Le agregamos la ultima capa
num_classes = lsa_train_df['class'].nunique()

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Cargamos solo las capas compartidas con el modelo que entrenamos antes
model.load_weights("inceptionv3_hand_features.weights.h5", skip_mismatch=True)

# Lo compilamos, ahora con un learning rate un poco mas alto.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Y entrenamos
model.fit(lsa_train_ds, validation_data=lsa_val_ds, epochs=10)

Epoch 1/10


/home/octavio/materias/ia/tp_decision_trees/ia_venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:648: UserWarning: A total of 1 objects could not be loaded. Example error message for object <Dense name=dense_11, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(1024, 16), Received: value.shape=(1024, 36). Target variable: <Variable path=dense_11/kernel, shape=(1024, 16), dtype=float32, value=[[-0.02250922 -0.02932912  0.0746019  ...  0.02493007 -0.06474213
  -0.03626624]
 [-0.03921478 -0.01528396 -0.03929701 ... -0.0538302  -0.05247301
  -0.0546594 ]
 [-0.02866591  0.02207758  0.05660126 ...  0.05759862  0.0455526
  -0.01739256]
 ...
 [ 0.03058823  0.06481682  0.06096706 ...  0.06537391 -0.02342306
   0.03972448]
 [-0.01540674 -0.02526372  0.04881488 ...  0.04961404 -0.0309973
  -0.07006078]
 [-0.01786763 -0.01516022  0.05429    ... -0.02224328  0.03768231
  -0.01313288]]>

List of obj

20/20 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.1082 - loss: 3.2995 - val_accuracy: 0.4875 - val_loss: 1.7997
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.4875 - loss: 1.6609 - val_accuracy: 0.5688 - val_loss: 1.2980
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.6298 - loss: 1.1986 - val_accuracy: 0.7125 - val_loss: 0.9809
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.7540 - loss: 0.8374 - val_accuracy: 0.7000 - val_loss: 0.9027
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.7662 - loss: 0.7185 - val_accuracy: 0.7125 - val_loss: 0.8306
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8582 - loss: 0.5472 - val_accuracy: 0.8062 - val_loss: 0.7289
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8784 - loss: 0.4445 - val_accuracy: 0.7437 - val_loss: 0.7134
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.9121 - loss: 0.3536 - val_accuracy: 0.7437 - val_loss: 0.7005
Epo

Ahora definimos la que sera nuestra ultima capa de manera que clasifique sobre la cantidad de clases que nos interesa. 

In [25]:
# Definimos la ultima capa para que prediga acorde a nuestras clases
num_classes = lsa_train_df['class'].nunique()
print(f"Número de clases: {num_classes}")

x = base_model.output  # x es la salida del modelo hasta ahora
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularización para evitar overfitting

# Y creamos una nueva capa de salida que tome como input a la anterior y clasifique en num_classes clases
predictions = Dense(num_classes, activation='softmax')(x)  


model = Model(inputs=base_model.input, outputs=predictions)

Número de clases: 16


Configuramos el modelo usando momento adaptativo y *sparse categorial cross-entropy* pues es adecuada para clasificacion multiclase con enteros según la documentación.

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Para etiquetas enteras
    metrics=['accuracy']
)

Entrenemos el modelo con los nuevos datos

In [10]:
model.fit(train_ds, epochs = 10, validation_data = val_ds, verbose = 2, batch_size = 20) 

Epoch 1/10
20/20 - 51s - 3s/step - accuracy: 0.2328 - loss: 2.7291 - val_accuracy: 0.4812 - val_loss: 1.6282
Epoch 2/10
20/20 - 44s - 2s/step - accuracy: 0.5891 - loss: 1.3294 - val_accuracy: 0.5938 - val_loss: 1.2063
Epoch 3/10
20/20 - 45s - 2s/step - accuracy: 0.7047 - loss: 0.9117 - val_accuracy: 0.7063 - val_loss: 0.9157
Epoch 4/10
20/20 - 44s - 2s/step - accuracy: 0.7750 - loss: 0.7134 - val_accuracy: 0.7312 - val_loss: 0.8134
Epoch 5/10
20/20 - 44s - 2s/step - accuracy: 0.8406 - loss: 0.5348 - val_accuracy: 0.7625 - val_loss: 0.7187
Epoch 6/10
20/20 - 45s - 2s/step - accuracy: 0.8594 - loss: 0.4589 - val_accuracy: 0.7500 - val_loss: 0.7457
Epoch 7/10
20/20 - 45s - 2s/step - accuracy: 0.8906 - loss: 0.3802 - val_accuracy: 0.7875 - val_loss: 0.6368
Epoch 8/10
20/20 - 46s - 2s/step - accuracy: 0.9078 - loss: 0.3055 - val_accuracy: 0.8250 - val_loss: 0.6079
Epoch 9/10
20/20 - 46s - 2s/step - accuracy: 0.9047 - loss: 0.3032 - val_accuracy: 0.7625 - val_loss: 0.6331
Epoch 10/10
20/20 -

In [12]:
img_path = 'lsa16_segmented/1_1_1.png'  
img = image.load_img(img_path, target_size=(299, 299)) # La carga en img y le hace resize a 299x299
img_array = image.img_to_array(img)                    # La convierte a array de NumPy con dimensiones (299, 299, 3)
img_array = np.expand_dims(img_array, axis=0)          # Agrega una dimension mas al array, haciendolo (1, 299, 299, 3) para batching
img_array = preprocess_input(img_array)                # Matchea la representacion de la imagen a como la espera ImageNet (ej. mappea 0-255 a -1,1, cambia de RGB a BGR)

# Predict
predictions = model.predict(img_array)
decoded_predictions = decode_predictions(predictions, top=5)[0]

# Display results
plt.imshow(img)
plt.axis('off')
plt.show()

print("Top 5 Predictions:")
for i, (_, label, prob) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({prob * 100:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Received array with shape: (1, 16)